# Generative Adversarial Networks

In [1]:
# imports 
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
import numpy.random as rng

In [4]:
# neural network class
class neural_net(object):
    ''' Neural Network Class'''
    def __init__(self, layer_sizes, activationFn=tf.nn.relu):
        self.ip = tf.placeholder(shape=[None, layer_sizes[0]], dtype=tf.float32) # input to network
        self.activationFn = activationFn # network activation function
        self.weights = []
        self.biases = []
        for s1,s2 in zip(layer_sizes[0:-1], layer_sizes[1:]):
            self.weights += [tf.Variable(tf.truncated_normal(shape=[s1,s2], stddev=1./tf.sqrt(s2)))]
            self.biases += [tf.Variable(tf.zeros(shape= [s2]))]
    ''' Logits function'''
    def logits(self):
        h = self.ip   #initilaize layer with input
        ###  forward pass through network layers ###
        for w, b in zip(self.weights[:-1], self.biases[:-1]):
            h = self.activationFn(tf.matmul(h,w)+b)
        return tf.matmul(h, self.weights[-1])+tf.biases[-1]
            
            
            
        

In [5]:
c = neural_net()

TypeError: Value passed to parameter 'x' has DataType int32 not in list of allowed values: float16, float32, float64, complex64, complex128